# Beamsize minimization with Xopt

## define the diagnostic

In [ ]:
from camera import load_camera

In [ ]:
# setup camera and quads:
image_diagnostic = load_camera("EYG7")
q1_pv_name = "AWA:Bira3Ctrl:Ch03"
q2_pv_name = "AWA:Bira3Ctrl:Ch04"
q3_pv_name = "AWA:Bira3Ctrl:Ch05"
q1_range = [0,4.0]
q2_range = [-4.0,0]
q3_range = [0,4.0]

In [ ]:
image_diagnostic.test_measurement()

### Define the evaluator

In [ ]:
from epics import caput, caget
from xopt import Evaluator
import time
import numpy as np

IMAGE_CONSTRAINTS = {
            "bb_penalty": ["LESS_THAN", 0.0],
            "log10_total_intensity": ["GREATER_THAN", image_diagnostic.min_log_intensity]
        }
image_diagnostic.visualize = False
def evaluate_function(inputs: dict) -> dict:
    global image_diagnostic
    # caput valueslog_transform_acquisition_function
    for name, val in inputs.items():
        caput(name, val)

    # wait for changes to occur - use small wait time for interpolated measurements
    time.sleep(1)

    results = image_diagnostic.measure_beamsize(1, **inputs)

    # add total beam size
    sigma_xy = np.sqrt(np.array(results["Sx"])**2 + np.array(results["Sy"])**2)
    roundness = np.abs(np.array(results["Sx"]) - np.array(results["Sy"]))
    results["total_size"] = sigma_xy + roundness
    print(sigma_xy, roundness, results["total_size"], results["bb_penalty"])
    return results

evaluator = Evaluator(function=evaluate_function)

### Define VOCS
Here we define the names and ranges of input parameters, the names and settings of
objectives, and the names and settings of constraints. Note that the keys here should
 be referenced in the evaluate function above.

In [ ]:
from xopt import VOCS

# define control PVs and ranges here
vocs = VOCS(
    variables = {
        q1_pv_name: q1_range,
        q2_pv_name: q2_range,
        q3_pv_name: q3_range,
    },
    objectives = {"total_size":"MINIMIZE"},
    constraints = IMAGE_CONSTRAINTS
)

### Define the Generator

In [ ]:
from xopt.generators import get_generator
from xopt.generators.bayesian.models.standard import StandardModelConstructor

model_constructor = StandardModelConstructor(use_low_noise_prior=False)
generator = get_generator("expected_improvement")(
    vocs=vocs, gp_constructor=model_constructor, 
    n_interpolate_points=5, 
    turbo_controller="optimize",
    log_transform_acquisition_function=True,
)
generator.numerical_optimizer.max_time = 10.0
#generator.turbo_controller.failure_tolerance = 6
#generator.turbo_controller.success_tolerance = 6

###  Combine into Xopt object

In [ ]:
from xopt import Xopt
import yaml
save_image_location = '.'
dump_filename = save_image_location + "/beamsize_minimization.yml"
X = Xopt(vocs=vocs, generator=generator, evaluator=evaluator, dump_file=dump_filename)

In [ ]:
import pandas as pd
#X.add_data(pd.DataFrame(
#    yaml.safe_load(
#        open("beamsize_minimization.yml")
#    )["data"]
#    )
#)

## Introspection
Objects in Xopt can be printed to a string or dumped to a text file for easy
introspection of attributes and current configuration.

In [ ]:
# Convenient representation of the state.
X

### Initialization

In [ ]:
from epics import caget_many
from xopt.utils import get_local_region
import pandas as pd
# 0.684 	1.135 	0.146 0.5
X.evaluate_data(pd.DataFrame([{
    q1_pv_name: 1.14, # 
    q2_pv_name: -2.44, # 
    q3_pv_name: 1.41,
}], columns=X.vocs.variable_names,index=[0]))

# get current point
current_value = dict(zip(X.vocs.variable_names, caget_many(X.vocs.variable_names)))

# get small region around current point to sample
random_sample_region = get_local_region(current_value,X.vocs, fraction=0.05)

random_sample_region

In [ ]:
X.data

In [ ]:
# random sample in a local region
X.random_evaluate(10, custom_bounds=random_sample_region)

In [ ]:
# examine the data stored in Xopt
X.data

In [ ]:
# run optimization
for i in range(20):
    print(i)
    X.step()

In [ ]:
# visualize result
X.data.plot(y=X.vocs.objective_names[0])

In [ ]:
X.data.plot(y=X.vocs.variable_names)

In [ ]:
image_diagnostic.test_measurement()

In [ ]:
idx, val = X.vocs.select_best(X.data)

In [ ]:
X.evaluate_data(X.data.iloc[idx][X.vocs.variable_names])

In [ ]:
X.data.iloc[idx][X.vocs.variable_names].to_dict(orient="records")

In [ ]:
image_diagnostic.test_measurement()